In [11]:
import os
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("example") \
    .getOrCreate()


In [12]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=example>

# Data info
[link](https://dumps.wikimedia.org/other/pagecounts-raw/)

In [13]:
path = "file://" + os.getcwd() + "/ds/pagecounts-20100806-030000"


In [14]:

bookRDD = sc.textFile(path)

In [15]:
sample = bookRDD.take(10)

In [16]:
sample

['aa.b Main_Page 1 4881',
 'aa.b Special:Contributions/Beau 1 4938',
 'aa.b Special:WhatLinksHere/MediaWiki:Makesysopsubmit 1 4550',
 'aa.b User:Ahoerstemeier 1 4388',
 'aa.b User:Monobi 1 5500',
 'aa.d Special:Contributions/Les_Meloures 1 4949',
 'aa %D0%90%D1%80%D0%B3%D0%B0%D0%B7%D0%BC 1 655',
 'aa Amiga_600 1 655',
 'aa DJ_Dlee 1 653',
 'aa Main_Page 2 9494']

In [17]:
number_of_lines = bookRDD.count()
number_of_lines

4729148

In [18]:
enPages = bookRDD.filter(lambda x: x.startswith("en "))

In [19]:
enPages.count()

2245124

In [20]:
enPages.take(10)

['en ! 13 307446',
 'en !! 2 14565',
 'en !!! 23 356424',
 'en !!!Fuck_You!!! 6 58373',
 'en !!!Fuck_You!!!_and_Then_Some 1 10185',
 'en !!!_(album) 4 29794',
 'en !!Destroy-Oh-Boy!! 1 38670',
 'en !%3D 2 14116',
 'en !5_The_Blood 1 6176',
 'en !Action_Pact! 2 18003']

In [21]:
enPagesTuples = enPages.map(lambda x: tuple(x.split()))
enPagesTuples = enPagesTuples.map(lambda tupla: (tupla[0], tupla[1], int(tupla[2]), int(tupla[3])))

In [22]:
enPagesTuples.count()

2245124

In [23]:
enPagesTuplesSorted = enPagesTuples.sortBy(lambda x: x[3], ascending=False)

In [24]:
enPagesTuplesSorted.take(5)

[('en', 'Main_Page', 222302, 5025224410),
 ('en', 'United_States', 4694, 2160246570),
 ('en', 'United_Kingdom', 2164, 1138630820),
 ('en', 'Canada', 1753, 688248121),
 ('en', 'England', 1401, 626431448)]

In [25]:
enPagesTuples.max(key=lambda x: x[2])

('en', 'Special:Random', 405305, 218224631)

In [26]:
def print_histogram(hist_data):
    bin_edges, bin_counts = hist_data
    for bin_start, bin_end, count in zip(bin_edges[:-1], bin_edges[1:], bin_counts):
        print("[{: >8} - {: >8}): {}".format(bin_start, bin_end, '*' * count))


In [27]:
pageVisits = enPagesTuples.map(lambda x: x[2])

# Calculate the histogram with 20 bins
histogram_data = pageVisits.histogram(buckets=20)


In [29]:
histogram_data

([1.0,
  20266.2,
  40531.4,
  60796.600000000006,
  81061.8,
  101327.0,
  121592.20000000001,
  141857.4,
  162122.6,
  182387.80000000002,
  202653.0,
  222918.2,
  243183.40000000002,
  263448.60000000003,
  283713.8,
  303979.0,
  324244.2,
  344509.4,
  364774.60000000003,
  385039.8,
  405305],
 [2245119, 1, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [28]:
spark.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x7f5fd464c280>>